In [75]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [76]:
# Caminho do arquivo no projeto
pasta_arquivo = '/mnt/c/Users/Isabel/Documents/bootcamp_CDD/avanti-bootcamp-cdd/data/raw/diamonds.csv'

#Leitura do conjunto de dados 
df = pd.read_csv(pasta_arquivo, delimiter=',')
df.head(10)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


In [77]:
#Leitura do dicionário
pasta_dicionario = '/mnt/c/Users/Isabel/Documents/bootcamp_CDD/avanti-bootcamp-cdd/data/external/Dicionario_de_dados.csv'
dicionario = pd.read_csv(pasta_dicionario, delimiter=',')
dicionario.head(10)

,Variável,Tradução,Descrição,Espectro de opções,Tipo,Subtipo
0,Carat,Quilate (ct),Peso do diamante em quilates,0.2 ---- 5.01 ct,Quantitativo,Contínua
1,Cut,Corte,Qualidade do corte,"Fair, Good, Very Good, Premium, Ideal",Qualitativo,Ordinal
2,Color,Cor,Cor do diamante,D (best) - J (worst),Qualitativo,Ordinal
3,Clarity,Clareza,Clareza do diamante,"1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF ...",Qualitativo,Ordinal
4,Depth,Profundidade,Profundidade do diamante,Valor obtido pela equação ( percentual de prof...,Quantitativo,Contínua
5,Table,Proporção,Proporção base vs topo,43.0 - 95.0,Quantitativo,Contínua
6,Price,Preço,Valor do diamante em USD,"$326 ---- $18,823",Quantitativo,Discretas
7,X,Comprimento,Comprimento em milímetros (mm),0--10.74 mm,Quantitativo,Contínua
8,Y,Largura,Largura em milímetros(mm),0--58.9 mm,Quantitativo,Contínua
9,Z,Altura,Altura em milímetros (mm),0--31.8 mm,Quantitativo,Contínua


In [78]:
print(dicionario.columns)

Index(['Variável', 'Tradução', 'Descrição', 'Espectro de opções', 'Tipo',
       'Subtipo'],
      dtype='object')


In [79]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'price'

variavel_qualitativo = (
    dicionario
    .query("Subtipo == 'Ordinal' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variáveis qualitativo: ordinal")
print(variavel_qualitativo)
print(" ")
variavel_quanti_continua = (
    dicionario
    .query("Subtipo == 'Contínua' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")
variavel_quanti_discreta = (
    dicionario
    .query("Subtipo == 'Discretas' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: discreta")
print(variavel_quanti_discreta)
print(" ")

X = df.drop(columns=[variavel_alvo])
Y = df[variavel_alvo]

Variáveis qualitativo: ordinal
['Cut', 'Color', 'Clarity']
 
Variável quantitativo: Contínua
['Carat', 'Depth', 'Table', 'X', 'Y', 'Z']
 
Variável quantitativo: discreta
['Price']
 


In [80]:
processamento_ordinal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), #Tratamento de dados faltantes com substituição pela MODA
    ("encoding", OrdinalEncoder()), #Codificação de variáveis
     
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreta = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])

In [81]:
pre_processador = ColumnTransformer([
    ("ordinal", processamento_ordinal, variavel_qualitativo),
    ("continuo", processamento_continuos, variavel_quanti_continua),
    ("discreta", processamento_discreta, variavel_quanti_discreta)
])